In [12]:
from dataclasses import dataclass
import torch
from torch import nn
from torch.nn import functional as F
import math
from transformers import GPT2LMHeadModel

In [13]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.shape

        qkv =  self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # att = q @ k.transpose(-2, -1) * (1.0 / math.sqrt(k.shape[-1]))
        # att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        # att = F.softmax(att, dim=-1)
        # y = att @ v

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head:int = 12
    n_embd: int = 768

In [14]:
import inspect


class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        assert T <= self.config.block_size, f"Cannot foward sequence of length {T} > block size {self.config.block_size}"

        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = pos_emb + tok_emb

        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024),
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280),
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600),
        }[model_type]
        config_args['vocab_size'] = 50257
        config_args['block_size'] = 1024

        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]

        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # print(f"sd_hf[{k}].shape = {sd_hf[k].shape}")
                # print(f"sd[{k}].shape = {sd[k].shape}")
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, device):
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}

        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and "cuda" in device
        print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

In [18]:
import tiktoken
import os
import numpy as np

def load_tokens(filename):
    npt = np.load(filename)
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt

class DataLoaderLite:
    def __init__(self, B, T, split):
        self.B = B
        self.T = T
        assert split in {'train', 'val'}

        data_root = "/kaggle/input/fineweb-edu-1gb"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"

        self.reset()
        
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position + B*T + 1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += B*T

        if self.current_position + (B*T+1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = 0
            
        return x, y

    def reset(self):
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = 0

In [26]:
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 101

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr

    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    # coeff = 0.5 * math.cos(math.pi * decay_ratio / 2)
    return min_lr + coeff * (max_lr - min_lr)

In [23]:
import time
from collections import OrderedDict

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

total_batch_size = 131072
B = 8
T = 1024
assert total_batch_size % (B*T) == 0
grad_accum_steps = total_batch_size // (B*T)
print(f"total desired batch size: {total_batch_size}")
print(f"=> calculated gradient accumulation steps: {grad_accum_steps}")


train_loader = DataLoaderLite(B, T, split="train")
val_loader = DataLoaderLite(B, T, split="val")

torch.set_float32_matmul_precision("high")

model = GPT(GPTConfig(vocab_size=50304))
model_state_dict = torch.load("/kaggle/input/gpt2_fineweb_100m_weights_3/pytorch/default/1/gpt2_fineweb_100M_weights_3.pth", map_location=device)
model_state_dict = OrderedDict({key.lstrip('_orig_mod.'): value for key, value in model_state_dict.items()})
model.load_state_dict(model_state_dict)


total desired batch size: 131072
=> calculated gradient accumulation steps: 16


<All keys matched successfully>

In [24]:
model.to(device)
model = torch.compile(model)

In [27]:
from tqdm import tqdm

enc = tiktoken.get_encoding("gpt2")
optimizer = model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device=device)

iters = []
loss_accum_list = []

for step in tqdm(range(max_steps), desc="Training model...", ncols=120):
    t0 = time.time()
    loss_accum = 0.0
    val_loss = 0.0
    if step % 20 == 0:
        model.eval()
        val_loader.reset()
        with torch.no_grad():
            x, y = val_loader.next_batch()
            x, y = x.to(device), y.to(device)
            logits, val_loss = model(x, y)
        tqdm.write(f"val loss: {val_loss.item():.4f}")
        tqdm.write("")
            
    model.train()
    # x, y = train_loader.next_batch()
    # x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    for micro_step in tqdm(range(grad_accum_steps), desc="Gradient accumulation...", ncols=80):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        
        # with torch.autocast(device_type=device, dtype=torch.bfloat16):
        #     logits, loss = model(x, y)
        logits, loss = model(x, y)
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        loss.backward()
    # with torch.autocast(device_type=device, dtype=torch.bfloat16):
    #     logits, loss = model(x, y)
    # logits, loss = model(x, y)
    # loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
    optimizer.step()
    # torch.cuda.synchronize()
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1-t0)

    iters.append(step)
    loss_accum_list.append(loss_accum.detach())
    
    
    if step % 5 == 0 or step == max_steps-1:
        tqdm.write(f"step{step}, loss_accum: {loss_accum.item():.3f} | lr: {lr:.4e} | norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")


    if (step > 0 and step % 250 == 0) or step == max_steps-1:
        tqdm.write("")
        model.eval()
        num_return_sequences = 4
        max_length = 32
        tokens = enc.encode("Hello, I'm a language model,")
        tokens = torch.tensor(tokens, dtype=torch.long)
        tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
        xgen = tokens.to(device)
        sample_rng = torch.Generator(device=device)
        sample_rng.manual_seed(42)
        while xgen.size(1) < max_length:
            # forward the model to get the logits
            with torch.no_grad():
                # with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                logits, loss = model(xgen) # (B, T, vocab_size)
                # take the logits at the last position
                logits = logits[:, -1, :] # (B, vocab_size)
                # get the probabilities
                probs = F.softmax(logits, dim=-1)
                topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
                ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
                xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
                xgen = torch.cat((xgen, xcol), dim=1)
        for i in range(num_return_sequences):
            tokens = xgen[i, :max_length].tolist()
            decoded = enc.decode(tokens)
            tqdm.write(f"sample {i}: {decoded}")
        tqdm.write("")

    os.system('clear')

num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True


Training model...:   0%|                                                                        | 0/101 [00:01<?, ?it/s]

val loss: 5.7001




Training model...:   1%|▌                                                             | 1/101 [00:41<1:09:05, 41.46s/it]

step0, loss_accum: 5.586 | lr: 6.0000e-05 | norm: 0.3065 | dt: 40111.33ms | tok/sec: 204.23


...

val loss: 5.4643




Training model...:  99%|███████████████████████████████████████████████████████████▍| 100/101 [1:03:54<00:37, 37.79s/it]

step100, loss_accum: 5.401 | lr: 6.0161e-05 | norm: 0.2072 | dt: 38559.19ms | tok/sec: 212.45



Training model...: 100%|████████████████████████████████████████████████████████████| 101/101 [1:04:02<00:00, 38.05s/it]

sample 0: Hello, I'm a language model, and I'll be a big, then just another person by a big problem in the next moment I will make a big
sample 1: Hello, I'm a language model, just it is a very fast and to be able to determine and how those people can get it to look up to make
sample 2: Hello, I'm a language model, but I see a team on the case of research.
We’ve got to see the computer, and more
sample 3: Hello, I'm a language model, the only three times.
- As the image, we now say, the end of the new data.
So



In [28]:
torch.save(model.state_dict(), "/kaggle/working/gpt2_fineweb_100M_weights_4.pth")
print("model weights saved to /kaggle/working/gpt2_fineweb_100M_weights_4.pth")

model weights saved to /kaggle/working/gpt2_fineweb_100M_weights_4.pth


In [ ]:
from IPython.display import FileLink

# Make sure this matches the exact filename and path you used when saving your model
FileLink('/kaggle/working/gpt2_fineweb_100M_weights_3.pth')

### __fineweb dataset__

In [ ]:
# """
# FineWeb-Edu dataset (for srs pretraining)
# https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu
# Downloads and tokenizes the data and saves data shards to disk.
# Run simply as:
# $ python fineweb.py
# Will save shards to the local directory "edu_fineweb10B".
# """

# import os
# import multiprocessing as mp
# import numpy as np
# import tiktoken
# from datasets import load_dataset # pip install datasets
# from tqdm import tqdm # pip install tqdm

# # ------------------------------------------
# local_dir = "edu_fineweb10B"
# remote_name = "sample-10BT"
# shard_size = int(1e8) # 100M tokens per shard, total of 100 shards
# MAX_SHARDS_TO_GENERATE = 5 # <--- ADDED: Limit to 10 shards

# # create the cache the local directory if it doesn't exist yet
# local_dir = "fineweb_dataset"
# DATA_CACHE_DIR = os.path.join("/kaggle/working/", local_dir)
# os.makedirs(DATA_CACHE_DIR, exist_ok=True)

# print(f"Saving data shards to: {DATA_CACHE_DIR}")
# print(f"Each shard will contain approximately {shard_size / 1e6:.1f} M tokens.")
# print(f"Generating a maximum of {MAX_SHARDS_TO_GENERATE} shards.")


# # download the dataset
# print(f"Loading dataset 'HuggingFaceFW/fineweb-edu' with name '{remote_name}'...")
# fw = load_dataset("HuggingFaceFW/fineweb-edu", name=remote_name, split="train")
# print("Dataset loaded successfully.")

# # init the tokenizer
# enc = tiktoken.get_encoding("gpt2")
# eot = enc._special_tokens['<|endoftext|>'] # end of text token
# def tokenize(doc):
#     # tokenizes a single document and returns a numpy array of uint16 tokens
#     tokens = [eot] # the special <|endoftext|> token delimits all documents
#     tokens.extend(enc.encode_ordinary(doc["text"]))
#     tokens_np = np.array(tokens)
#     assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token dictionary too large for uint16"
#     tokens_np_uint16 = tokens_np.astype(np.uint16)
#     return tokens_np_uint16

# def write_datafile(filename, tokens_np):
#     np.save(filename, tokens_np)
#     print(f"Saved {filename} with {len(tokens_np)} tokens.")


# # tokenize all documents and write output shards, each of shard_size tokens (last shard has remainder)
# nprocs = max(1, os.cpu_count()//2)
# print(f"Using {nprocs} multiprocessing cores for tokenization.")

# with mp.Pool(nprocs) as pool:
#     shard_index = 0
#     # preallocate buffer to hold current shard
#     all_tokens_np = np.empty((shard_size,), dtype=np.uint16)
#     token_count = 0
#     progress_bar = None
    
#     # Iterate through tokenized documents
#     for tokens in pool.imap(tokenize, fw, chunksize=16):
#         # Check if we have reached the maximum number of shards
#         if shard_index >= MAX_SHARDS_TO_GENERATE:
#             print(f"Reached maximum of {MAX_SHARDS_TO_GENERATE} shards. Stopping.")
#             break # Exit the loop

#         # is there enough space in the current shard for the new tokens?
#         if token_count + len(tokens) < shard_size:
#             # simply append tokens to current shard
#             all_tokens_np[token_count:token_count+len(tokens)] = tokens
#             token_count += len(tokens)
#             # update progress bar
#             if progress_bar is None:
#                 progress_bar = tqdm(total=shard_size, unit="tokens", desc=f"Shard {shard_index}")
#             progress_bar.update(len(tokens))
#         else:
#             # write the current shard and start a new one
#             split = "val" if shard_index == 0 else "train" # First shard can be val, rest train
#             filename = os.path.join(DATA_CACHE_DIR, f"edufineweb_{split}_{shard_index:06d}")
            
#             # split the document into whatever fits in this shard; the remainder goes to next one
#             remainder = shard_size - token_count
#             progress_bar.update(remainder)
#             all_tokens_np[token_count:token_count+remainder] = tokens[:remainder]
#             write_datafile(filename, all_tokens_np)
            
#             if progress_bar is not None:
#                 progress_bar.close() # Close the current progress bar
            
#             shard_index += 1
            
#             # Check again after incrementing shard_index
#             if shard_index >= MAX_SHARDS_TO_GENERATE:
#                 print(f"Reached maximum of {MAX_SHARDS_TO_GENERATE} shards. Stopping.")
#                 # Populate the next shard with the leftovers of the current doc, which will be the last one
#                 all_tokens_np[0:len(tokens)-remainder] = tokens[remainder:]
#                 token_count = len(tokens)-remainder
#                 break # Exit the loop immediately after writing the 10th shard

#             progress_bar = None # Reset progress bar for the next shard
#             # populate the next shard with the leftovers of the current doc
#             all_tokens_np[0:len(tokens)-remainder] = tokens[remainder:]
#             token_count = len(tokens)-remainder

#     # write any remaining tokens as the last shard, only if we didn't explicitly stop due to MAX_SHARDS_TO_GENERATE
#     if token_count != 0 and shard_index < MAX_SHARDS_TO_GENERATE:
#         split = "val" if shard_index == 0 else "train"
#         filename = os.path.join(DATA_CACHE_DIR, f"edufineweb_{split}_{shard_index:06d}")
#         write_datafile(filename, all_tokens_np[:token_count])

#     # Ensure the last progress bar is closed if it was open
#     if progress_bar is not None:
#         progress_bar.close()

# print("Data processing complete.")